In [75]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [63]:
sales_data = pd.read_csv('sales_data.csv',parse_dates=['OrderDate'],dtype={'_CustomerID':'str'})

In [64]:
sales_data.dtypes

OrderNumber                 object
Sales Channel               object
WarehouseCode               object
ProcuredDate                object
OrderDate           datetime64[ns]
ShipDate                    object
DeliveryDate                object
CurrencyCode                object
_SalesTeamID                 int64
_CustomerID                 object
_StoreID                     int64
_ProductID                   int64
Order Quantity               int64
Discount Applied           float64
Unit Price                 float64
Unit Cost                  float64
dtype: object

In [65]:
df = sales_data.copy()

In [66]:
df.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost
0,SO - 000101,In-Store,WARE-UHY1004,12/31/2017,2018-05-31,6/14/2018,6/19/2018,USD,6,15,259,12,5,0.075,1963.1,1001.181
1,SO - 000102,Online,WARE-NMK1003,12/31/2017,2018-05-31,6/22/2018,7/2/2018,USD,14,20,196,27,3,0.075,3939.6,3348.660
2,SO - 000103,Distributor,WARE-UHY1004,12/31/2017,2018-05-31,6/21/2018,7/1/2018,USD,21,16,213,16,1,0.050,1775.5,781.220
3,SO - 000104,Wholesale,WARE-NMK1003,12/31/2017,2018-05-31,6/2/2018,6/7/2018,USD,28,48,107,23,8,0.075,2324.9,1464.687
4,SO - 000105,Distributor,WARE-NMK1003,4/10/2018,2018-05-31,6/16/2018,6/26/2018,USD,22,49,111,26,8,0.100,1822.4,1476.144


In [67]:
df['Revenue'] = ((df['Unit Price'] - (df['Unit Price']*df['Discount Applied'])) - df['Unit Cost']) * df['Order Quantity']

In [70]:
columns = ['OrderNumber','_CustomerID','OrderDate','Revenue']
rfm_data = df[columns]

In [71]:
rfm_data

,OrderNumber,_CustomerID,OrderDate,Revenue
0,SO - 000101,15,2018-05-31,4073.4325
1,SO - 000102,20,2018-05-31,886.4100
2,SO - 000103,16,2018-05-31,905.5050
3,SO - 000104,48,2018-05-31,5486.7640
4,SO - 000105,49,2018-05-31,1312.1280
...,...,...,...,...
7986,SO - 0008087,41,2020-12-30,94.9725
7987,SO - 0008088,29,2020-12-30,6725.4600
7988,SO - 0008089,32,2020-12-30,1338.9950
7989,SO - 0008090,42,2020-12-30,1286.4000


In [77]:
max_order_date = max(df['OrderDate'])

today = max_order_date + timedelta(days=2)

In [80]:
rfm = rfm_data.groupby('_CustomerID').agg({'OrderDate': lambda x: (today-x.max()).days,
                                     'OrderNumber':'count','Revenue':'sum'})

In [81]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']

In [83]:
r = pd.qcut(rfm['Recency'],q=5,labels = range(5,0,-1))
f = pd.qcut(rfm['Frequency'],q=5,labels = range(1,6))
m = pd.qcut(rfm['Monetary'],q=5,labels = range(1,6))

In [84]:
rfm_df = rfm.assign(R=r.values,F=f.values,M=m.values)
rfm_df

,Recency,Frequency,Monetary,R,F,M
_CustomerID,,,,,,
1,9,152,335933.6115,2,2,1
10,15,158,435122.1870,1,3,3
11,6,178,487614.2415,3,5,5
12,3,210,616719.2550,5,5,5
13,4,171,441003.2795,4,4,3
14,5,157,381450.0280,3,3,2
15,4,142,441668.3550,4,1,4
16,3,135,402938.7705,5,1,2
17,6,175,534027.3860,3,5,5


In [85]:
rfm_df['rfm_group'] = rfm_df[['R','F','M']].apply(lambda x: '-'.join(x.astype(str)),axis=1)
rfm_df['rfm_score'] = rfm_df[['R','F','M']].sum(axis=1)

In [88]:
rfm_df.sort_values('rfm_score',ascending=False).reset_index()

,_CustomerID,Recency,Frequency,Monetary,R,F,M,rfm_group,rfm_score
0,12,3,210,616719.2550,5,5,5,5-5-5,15
1,29,2,179,531770.6920,5,5,5,5-5-5,15
2,21,3,164,479383.0905,5,4,5,5-4-5,14
3,19,3,165,443231.8335,5,4,4,5-4-4,13
4,17,6,175,534027.3860,3,5,5,3-5-5,13
5,11,6,178,487614.2415,3,5,5,3-5-5,13
6,32,2,173,435206.0710,5,5,3,5-5-3,13
7,39,4,176,471608.1425,4,5,4,4-5-4,13
8,4,5,167,526981.0630,3,4,5,3-4-5,12
9,18,6,186,451637.7540,3,5,4,3-5-4,12
